## baidu 翻译 API

In [5]:
import requests
import random
import json
from hashlib import md5


def en2zh(query):
    query = query.strip()
    if query is None or len(query) == 0:
        return ""
    
    # Set your own appid/appkey.
    appid = '20220720001277966'
    appkey = 'nN9pUmkp8Sm8skkanHMu'

    # For list of language codes, please refer to `https://api.fanyi.baidu.com/doc/21`
    from_lang = 'en'
    to_lang =  'zh'

    endpoint = 'http://api.fanyi.baidu.com'
    path = '/api/trans/vip/translate'
    url = endpoint + path
    
    # Generate salt and sign
    def make_md5(s, encoding='utf-8'):
        return md5(s.encode(encoding)).hexdigest()

    salt = random.randint(32768, 65536)
    sign = make_md5(appid + query + str(salt) + appkey)

    # Build request
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    payload = {'appid': appid, 'q': query, 'from': from_lang, 'to': to_lang, 'salt': salt, 'sign': sign}

    try:
        # Send request
        r = requests.post(url, params=payload, headers=headers)
        res = r.json()
        trans_res = res['trans_result']
        temp = []
        for tran in trans_res:
            temp.append(tran['dst'])
        final_res = "\n".join(temp)
    except Exception as e:
        print(e)
        final_res = "Error Translate"
    return final_res

In [6]:
result = en2zh("Rewrite the sentence into more descriptive form:\nThe food was delicious.")

'trans_result'


In [7]:
result

'Error Translate'

## Helsinki-NLP/opus-mt-en-zh
英-中翻译模型

In [1]:
from transformers import pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-zh", device=0)

/opt/conda/lib/python3.7/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
import re
def en2zh_by_model(query):
    query = query.strip()
    if len(query) >= 350:
        return query
    if not query:
        return ""
    # input 中含有代码片段
    if re.search("^def.*", query):
        return query
    if re.search("^Class.*", query):
        return query
    # 当一句话中有 \n 时，需要拆开喂到模型，最后再用 \n 拼接
    q_list = query.split('\n')
    temp = []
    for q in q_list:
        temp.append(translator(q)[0]['translation_text'])
    res = "\n".join(temp)
    return res

In [6]:
en2zh_by_model("this is a test, 7x * 8 = y")

'这是一个测试, 7x * 8=y'

In [7]:
import pandas as pd
raw_data = pd.read_json('./alpaca_data.json')

In [8]:
raw_data

,instruction,input,output
0,Give three tips for staying healthy.,,1.Eat a balanced diet and make sure to include...
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye..."
2,Describe the structure of an atom.,,"An atom is made up of a nucleus, which contain..."
3,How can we reduce air pollution?,,There are a number of ways to reduce air pollu...
4,Describe a time when you had to make a difficu...,,I had to make a difficult decision when I was ...
...,...,...,...
51997,Generate an example of what a resume should li...,,"Jean Tremaine\n1234 Main Street, Anytown, CA 9..."
51998,Arrange the items given below in the order to ...,"cake, me, eating",I eating cake.
51999,Write an introductory paragraph about a famous...,Michelle Obama,Michelle Obama is an inspirational woman who r...
52000,Generate a list of five things one should keep...,,1. Research potential opportunities and carefu...


In [5]:
from tqdm import tqdm
tqdm.pandas(desc='translate')
temp = raw_data.instruction.progress_map(lambda x: en2zh_by_model(x))

translate:   0%|          | 10/52002 [00:00<1:33:36,  9.26it/s]/opt/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:1048: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
translate: 100%|██████████| 52002/52002 [1:51:17<00:00,  7.79it/s]  


In [122]:
temp.to_json('./temp/zh_instruction.json', orient='records', force_ascii=False)

In [10]:
from tqdm import tqdm
tqdm.pandas(desc='translate')
temp = raw_data.input.progress_map(lambda x: en2zh_by_model(x))
temp.to_json('./temp/zh_input.json', orient='records', force_ascii=False)

translate:   0%|          | 37/52002 [00:00<17:28, 49.56it/s]/opt/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:1048: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
translate: 100%|██████████| 52002/52002 [59:53<00:00, 14.47it/s]  


In [ ]:
from tqdm import tqdm
tqdm.pandas(desc='translate')
temp = raw_data.output.progress_map(lambda x: en2zh_by_model(x))
temp.to_json('./temp/zh_output.json', orient='records', force_ascii=False)

translate:   0%|          | 0/52002 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:1048: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
translate:  31%|███       | 16110/52002 [53:16<2:15:15,  4.42it/s] 

# Merge multi files

In [13]:
from pathlib import Path
import pandas as pd

In [23]:
data_root = Path('./temp')
for i in data_root.glob('*.json'):
    print(i.stem)

zh_input
zh_instruction
zh_output


In [46]:
merged_df = pd.DataFrame()
for i in data_root.glob('*.json'):
    print(f"current file: {i}, stem: {i.stem}")
    temp = pd.read_json(i)
    temp.columns = [i.stem]
    merged_df= pd.concat([temp, merged_df], axis=1)

current file: temp/zh_input.json, stem: zh_input
current file: temp/zh_instruction.json, stem: zh_instruction
current file: temp/zh_output.json, stem: zh_output


In [47]:
merged_df

,zh_output,zh_instruction,zh_input
0,"1. 均衡饮食,确保包括大量水果和蔬菜。\n2. 定期锻炼,保持身体的活力和强壮。\n3. ...",给三个忠告 保持健康。,
1,三种主要颜色是红色、蓝色和黄色。,三种主要颜色是什么?,
2,"原子由核组成,核中包含质子和中子,周围环绕着围绕核轨道运行的电子。质子和中子有正电荷,而电子...",描述原子的结构。,
3,There are a number of ways to reduce air pollu...,我们如何减少空气污染?,
4,I had to make a difficult decision when I was ...,描述你不得不作出困难决定的时间。,
...,...,...,...
51997,"Jean Tremaine\n1234 Main Street, Anytown, CA 9...","生成一个实例, 说明烹饪专业的 履历会想要什么 。",
51998,我吃蛋糕。,"将下列项目排列如下,以完成以下句子:","蛋糕,我,吃饭"
51999,Michelle Obama is an inspirational woman who r...,写一段关于名人的介绍性段落。,米歇尔·奥巴马
52000,1. 研究潜在机会并认真考虑各种备选办法。\n2. 确保新的职业真正适合。\n3. 了解职业...,"在考虑职业变化时,要提出一份五件事的清单。",


In [ ]:
zh_output = merged_df.pop('zh_output')

In [56]:
merged_df.insert(2, 'zh_output', zh_output)

In [57]:
merged_df

,zh_instruction,zh_input,zh_output
0,给三个忠告 保持健康。,,"1. 均衡饮食,确保包括大量水果和蔬菜。\n2. 定期锻炼,保持身体的活力和强壮。\n3. ..."
1,三种主要颜色是什么?,,三种主要颜色是红色、蓝色和黄色。
2,描述原子的结构。,,"原子由核组成,核中包含质子和中子,周围环绕着围绕核轨道运行的电子。质子和中子有正电荷,而电子..."
3,我们如何减少空气污染?,,There are a number of ways to reduce air pollu...
4,描述你不得不作出困难决定的时间。,,I had to make a difficult decision when I was ...
...,...,...,...
51997,"生成一个实例, 说明烹饪专业的 履历会想要什么 。",,"Jean Tremaine\n1234 Main Street, Anytown, CA 9..."
51998,"将下列项目排列如下,以完成以下句子:","蛋糕,我,吃饭",我吃蛋糕。
51999,写一段关于名人的介绍性段落。,米歇尔·奥巴马,Michelle Obama is an inspirational woman who r...
52000,"在考虑职业变化时,要提出一份五件事的清单。",,1. 研究潜在机会并认真考虑各种备选办法。\n2. 确保新的职业真正适合。\n3. 了解职业...


In [67]:
merged_df.sample(5)

,zh_instruction,zh_input,zh_output
32221,概述了一篇关于气候变化的文章。,Climate change is a growing global phenomenon ...,"气候变化已经是一个全球性问题,已经造成了伤害,预计会随着时间推移而恶化。 它正在造成更极端的..."
34852,"产生一个10个字句,用“草莓”一词。",,甜甜而酸的草莓 充满了空气中的芳香味
23800,"建议采取一些战略,帮助运动员制定适当的目标。",,1. Set goals that are both challenging and ach...
22826,"列出3种优势,即使用GPT模型生成文本。",,The advantages of using a GPT model for text g...
22636,以黑体标明代名词的前奏 。,保罗和利兹去散步了 他们喜欢看风景,"保罗和丽兹 - ""他们""的前身"


In [76]:
merged_df.columns = ['instruction', 'input', 'output']

In [79]:
merged_df.head(5)

,instruction,input,output
0,给三个忠告 保持健康。,,"1. 均衡饮食,确保包括大量水果和蔬菜。\n2. 定期锻炼,保持身体的活力和强壮。\n3. ..."
1,三种主要颜色是什么?,,三种主要颜色是红色、蓝色和黄色。
2,描述原子的结构。,,"原子由核组成,核中包含质子和中子,周围环绕着围绕核轨道运行的电子。质子和中子有正电荷,而电子..."
3,我们如何减少空气污染?,,There are a number of ways to reduce air pollu...
4,描述你不得不作出困难决定的时间。,,I had to make a difficult decision when I was ...


In [78]:
merged_df.to_json('./temp/zh_alpaca_from_translate.json', orient='records', force_ascii=False)

# Merge Other files(e.g. from call openai)